In [1]:
import os
os.chdir('/home/gfan/gen-t-demo/code/')
os.getcwd()

'/home/gfan/gen-t-demo/code'

In [2]:
from IPython.display import display
# Parameters
benchmark = "t2d_gold"
source_table_name = "table_471_1.csv" # table_471, table_471_1 (with nulls injected), table_594, table_640
# benchmark = "tptr"
# source_table_name = "psql_0_c_oj_asia.csv" # psql_0_c_oj_asia, psql_8_s_ij_ps
BENCHMARKPATH = '/home/gfan/Datasets/%s/' % (benchmark)  
sim_threshold = 0.2 # for set similarity
integration_timeout = 3600 # for integration

In [3]:
# Find Candidate Tables
import sys
sys.path.append("discovery/")
from discovery_utils import get_lake, get_starmie_candidates
from discover_candidates import CandidateTables

runStarmie = 0
lake_dfs, all_lake_table_cols = get_lake(benchmark)
source_candidates = []
if runStarmie: 
    starmie_candidates = get_starmie_candidates(benchmark)
    source_candidates = starmie_candidates[source_table_name.replace('.csv', '')]

# Call CandidateTables to find candidates
candidate_table_finder = CandidateTables(benchmark, sim_threshold,lake_dfs, all_lake_table_cols, source_candidates)
candidateTablesFound, noCandidates = candidate_table_finder.find_candidates(source_table_name)
if noCandidates: 
    print(f"There were no candidates found (no candidate for key) for Source {source_table_name}")
else:
    print(f"{len(candidateTablesFound)} Candidate Tables found for Source Table {source_table_name}")
    print(list(candidateTablesFound.keys()))

Source table table_471_1.csv has shape (100, 5)
... Loaded in 514 data lake tables in 0.00 seconds


discovery/discover_candidates.py:94: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df = df.loc[np.bitwise_or.reduce(conditions, dtype=object)[0]].drop_duplicates().reset_index(drop=True)


12 Candidate Tables found for Source Table table_471_1.csv
['table_536.csv', 'table_50.csv', 'table_187.csv', 'table_267.csv', 'table_594.csv', 'table_25.csv', 'table_244.csv', 'table_95.csv', 'table_640.csv', 'table_292.csv', 'table_674.csv', 'table_760.csv']


In [4]:

# Prune Candidate Tables to find the set of Originating Tables
from prune_candidates import OriginatingTables
# Call OriginatingTables to prune candidates to a set of originating tables
originating_tables_finder = OriginatingTables(benchmark, candidateTablesFound)
originating_tables, matrix_traversal_runtimes = originating_tables_finder.find_originating_tables(source_table_name)
source_df = originating_tables_finder.source_table
primary_key = originating_tables_finder.primary_key
print(f"From {len(candidateTablesFound)} candidate tables, found {len(originating_tables)} originating tables")
print(originating_tables)
print(f"Matrix Initialization took {matrix_traversal_runtimes['matrix_initialization'][0]:.3f} seconds, Traversal took {matrix_traversal_runtimes['matrix_traversal'][0]:.3f} seconds")
origin_tables_matched_cols = {}
if originating_tables: origin_tables_matched_cols = {t: candidateTablesFound[t] for t in originating_tables}


/home/gfan/gen-t-demo/code/utils.py:66: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  selectedTuplesDf = df.loc[np.bitwise_and.reduce(conditions)[0]].drop_duplicates().reset_index(drop=True)


From 12 candidate tables, found 6 originating tables
['table_50.csv', 'table_267.csv', 'table_95.csv', 'table_674.csv', 'table_25.csv', 'table_244.csv']
Matrix Initialization took 1.903 seconds, Traversal took 0.157 seconds


In [5]:
import os
os.chdir('integration/')
os.getcwd()

'/home/gfan/gen-t-demo/code/integration'

In [6]:
import pandas as pd
def highlight_overlap_values(source_df, result_df, primary_key):
    ##  Highlight overlapping values in reproduced table
    # wrap overlapping values in astericks (*): f'**{x}**'
    # highlight in light blue: f'\033[1;36m{val}\033[0m'
    # highlight in blue: f'\033[1;34m{x}\033[0m'
    # highlight in red: f'\033[1;31m{x}\033[0m'
    # highlight in yellow: f'\033[1;33m{x}\033[0m'
    for rowIndx in result_df.index:
        key_val = result_df.iloc[rowIndx][primary_key]
        try: 
            source_row_indx = source_df.index[source_df[primary_key] == key_val].tolist()[0]
        except: continue
        for col in [col for col in result_df.columns if col in source_df.columns]:
            result_val = result_df.iloc[rowIndx][col]
            source_val = source_df.iloc[source_row_indx][col]
            if result_val == source_val or (pd.isna(result_val) and pd.isna(source_val)): 
                result_df.iloc[rowIndx,result_df.columns.get_loc(col)] = f'\033[1;33m{result_val}\033[0m'
            if not pd.isna(result_val) and pd.isna(source_val):
                result_df.iloc[rowIndx,result_df.columns.get_loc(col)] = f'\033[1;36m{result_val}\033[0m'
                
    return result_df

In [7]:
# Integrate set of originating tables to reproduce source table
from tabulate import tabulate
from targeted_integration import TableIntegration
import integration_utils as utils 
import sys
sys.path.append('../discovery/')
from evaluatePaths import setTDR

table_integrator = TableIntegration(benchmark, origin_tables_matched_cols, integration_timeout)
timed_out, noCandidates, numOutputVals = table_integrator.integrate_tables(source_table_name)
integration_result = table_integrator.reproducedSourceTable
table_ops = table_integrator.tableOpsUsed
TDR_recall, TDR_precision = setTDR(table_integrator.source_df, integration_result)
print(f"Final Integration Result has Recall {TDR_recall:.3f} and Precision {TDR_precision:.3f}")
# for op in table_ops:print(op)
integration_result = utils.order_rows_cols(source_df, primary_key, integration_result)
# integration_result_highlighted = utils.highlight_overlap_values_df(source_df, integration_result, primary_key)
# '''
# ==== Begin sample for paper
sample_integration_df = integration_result.loc[integration_result[primary_key].isin(['Pulp Fiction', 'Citizen Kane', 'Star Wars'])]
integration_result_highlighted = utils.highlight_overlap_values_df(source_df, sample_integration_df, primary_key) # for sample in paper
#  ''' 

Imported 6 Originating Tables:  ['table_50.csv', 'table_267.csv', 'table_95.csv', 'table_674.csv', 'table_25.csv', 'table_244.csv']


/home/gfan/gen-t-demo/code/integration/integration_utils.py:407: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  selectedTuples = df.loc[np.bitwise_and.reduce(conditions)[0]]
/home/gfan/gen-t-demo/code/integration/integration_utils.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[colname][i] = "null"+ str(null_count)
/home/gfan/miniconda3/envs/sdd/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

Finished Complementation in 0.00 sec
Finished Subsumption in 0.00 sec
Output tuples: ( total 124 )
Time taken FD algorithm: 0.54 sec
-----x---------x--------x---
Source Table has 100 rows, 5 columns
Resulting Table has 115 rows, 5 columns with value similarity of 0.970.
5 overlapping columns: ['Title', "Fans' Rank", 'Year', 'Director(s)', 'Overall Rank']
-----x---------x--------x---
Final Integration Result has Recall 0.850 and Precision 0.850


In [8]:
integration_result_highlighted

,Title,Fans' Rank,Year,Director(s),Overall Rank
0,Pulp Fiction,2,1994,Quentin Tarantino,15
1,Citizen Kane,3,1941,Orson Welles,2
2,Star Wars,5,1977,George Lucas,8


In [9]:
import numpy as np
expanded_integ_result, new_cols, new_tuples = table_integrator.expand_tables('all', size=5000)
# expanded_integration_result_highlighted = utils.highlight_overlap_values_df(source_df, expanded_integ_result, primary_key)
# '''
# ==== Begin sample for paper
sample_integ_df = expanded_integ_result.loc[expanded_integ_result[primary_key].isin(['Pulp Fiction', 'Citizen Kane', 'Star Wars'])]
sample_expand_df = expanded_integ_result.loc[expanded_integ_result[primary_key].isin(['Night of the Living Dead', 'Kill Bill: Vol. 2', 'O Brother, Where Art Thou?'])]
df = pd.concat([sample_integ_df,sample_expand_df]).drop('Rank', axis=1)
expanded_integration_result_highlighted = utils.highlight_overlap_values_df(source_df, df, primary_key) # for sample in paper
# '''
display(expanded_integration_result_highlighted)


Integrating raw Originating tables produces Table of shape (514, 8), with 4744 values
Integrated Table has 3 new columns (514 tuples): Index(['Religion', 'Actor', 'Rank'], dtype='object')
Integrated Table has 414 new tuples (8 columns)


/home/gfan/gen-t-demo/code/integration/integration_utils.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[colname][i] = "null"+ str(null_count)
/home/gfan/miniconda3/envs/sdd/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/gfan/gen-t-demo/code/integration/integration_utils.py:407: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify

,Title,Fans' Rank,Year,Director(s),Overall Rank,Religion,Actor
0,Pulp Fiction,2,1994,Quentin Tarantino,15,Atheist,John Travolta
1,Citizen Kane,3,1941,Orson Welles,2,nan,Orson Welles
2,Star Wars,5,1977,George Lucas,8,nan,nan
3,Kill Bill: Vol. 2,224,2004,Quentin Tarantino,701,nan,nan
4,Night of the Living Dead,229,1968,George A. Romero,139,Christian,nan
5,"O Brother, Where Art Thou?",232,2000,Joel Coen,776,Jewish,nan


In [10]:
ranked_df = table_integrator.rank_tuples(df, rankCol='Overall Rank')
ranked_df = utils.highlight_overlap_values_df(source_df, ranked_df, primary_key)
display(ranked_df)

,Title,Fans' Rank,Year,Director(s),Overall Rank,Religion,Actor
0,Citizen Kane,3,1941,Orson Welles,2,nan,Orson Welles
1,Star Wars,5,1977,George Lucas,8,nan,nan
2,Pulp Fiction,2,1994,Quentin Tarantino,15,Atheist,John Travolta
3,Night of the Living Dead,229,1968,George A. Romero,139,Christian,nan
4,Kill Bill: Vol. 2,224,2004,Quentin Tarantino,701,nan,nan
5,"O Brother, Where Art Thou?",232,2000,Joel Coen,776,Jewish,nan


In [11]:
filter_dict = {'Director(s)': ['Quentin Tarantino', 'Steven Spielberg'], 
               'Overall Rank': [i for i in range(1000)]}
filtered_df = table_integrator.filter_tuples(df, filter_dict)
filtered_df = utils.highlight_overlap_values_df(source_df, filtered_df, primary_key)
display(filtered_df)

,Title,Fans' Rank,Year,Director(s),Overall Rank,Religion,Actor
0,Pulp Fiction,2,1994,Quentin Tarantino,15,Atheist,John Travolta
1,Kill Bill: Vol. 2,224,2004,Quentin Tarantino,701,nan,nan


In [12]:
# Outer join list of candidate tables retrieved from data lake
# joined_df = utils.outerjoin(list(table_integrator.raw_tableDfs.values())) # originating tables
joined_df = utils.outerjoin(list(originating_tables_finder.candidate_table_dfs.values())) # candidate tables
joined_df = utils.order_rows_cols(source_df, primary_key, joined_df)
# joined_df = utils.highlight_overlap_values_df(source_df, joined_df, primary_key)
# '''
# ==== Begin sample for paper
joined_integ_df = joined_df.loc[joined_df[primary_key].isin(['Pulp Fiction', 'Citizen Kane', 'Star Wars'])]
joined_expand_df = joined_df.loc[joined_df[primary_key].isin(['Night of the Living Dead', 'Kill Bill: Vol. 2', 'O Brother, Where Art Thou?'])]
joined_df = pd.concat([joined_integ_df,joined_expand_df]).drop('Rank', axis=1)
joined_df = joined_df[df.columns]
joined_df = utils.highlight_overlap_values_df(source_df, joined_df, primary_key) # for sample in paper
# '''
display(joined_df)

,Title,Fans' Rank,Year,Director(s),Overall Rank,Religion,Actor
0,Pulp Fiction,nan,1994,nan,nan,nan,nan
1,Pulp Fiction,2,1994,Quentin Tarantino,15,Atheist,nan
2,Pulp Fiction,nan,1994,Quentin Tarantino,15,nan,John Travolta
3,Citizen Kane,3,1941,Orson Welles,2,nan,nan
4,Citizen Kane,nan,1941,nan,nan,nan,nan
5,Citizen Kane,nan,1941,Orson Welles,2,nan,Orson Welles
6,Star Wars,nan,1977,George Lucas,8,nan,nan
7,Star Wars,5,1977,George Lucas,8,nan,nan
8,Night of the Living Dead,229,1968,George A. Romero,139,Christian,nan
9,"O Brother, Where Art Thou?",232,2000,Joel Coen,776,Jewish,nan
